In [1]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

False

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

In [3]:
from langchain.chains import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.memory import ConversationBufferMemory

class Agent:
    def __init__(self, name, system_prompt, llm):
        self.name = name
        self.llm = llm
        self.system_prompt = system_prompt
        # Create a chat prompt that includes 'conversation_history'
        self.chat_prompt = ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template(self.system_prompt),
            # Include the conversation history and the last message in the prompt
            HumanMessagePromptTemplate.from_template("{conversation_history}\n{input}")
        ])
        self.chain = LLMChain(
            llm=self.llm,
            prompt=self.chat_prompt,
            verbose=False
        )

    def respond(self, input_text, conversation_history):
        response = self.chain.predict(
            input=input_text,
            conversation_history=conversation_history
        )
        return response

In [4]:
import os
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7,
    openai_api_key=os.getenv("OPENAI_API_KEY")
)


In [5]:
# Customer Proxy Agent
customer_proxy_agent = Agent(
    name="Customer_Proxy",
    system_prompt="You are a Customer Proxy Agent. Your role is to communicate customer's requirements to the Project Manager.",
    llm=llm
)

# Project Manager Agent
project_manager_agent = Agent(
    name="Project_Manager",
    system_prompt=(
        "You are a Project Manager Agent. Your role is to receive customer requirements and create a project plan based on outputs from each engineer. "
        "Coordinate with each engineering agent to ensure project milestones are met."
    ),
    llm=llm
)

# Requirement Engineer Agent
requirement_engineer_agent = Agent(
    name="Requirement_Engineer",
    system_prompt=(
        "You are a Requirement Engineer Agent. First, create the requirements document that outlines the use cases of the DevSecOps automation project requested by the project manager. "
        "\n\nAfter gathering requirements, calculate the effort to create the requirements document using the steps below:"
        "\n- Step 1: Estimate the total number of requirements needed."
        "\n- Step 2: Productivity rate = 5 requirements per day."
        "\n- Step 3: Effort = Work / Productivity rate."
    ),
    llm=llm
)

# System Engineer Agent
system_engineer_agent = Agent(
    name="System_Engineer",
    system_prompt=(
        "You are a System Engineer. Based on the requirements document, create a detailed design document that includes architecture for CI/CD pipeline automation, "
        "security integration, and infrastructure provisioning for the DevSecOps pipeline. "
        "Estimate the number of pages for the design elements and calculate the effort required using the following steps:"
        "\n- Step 1: Work = Estimate the total number of pages."
        "\n- Step 2: Productivity rate = 5 pages per day."
        "\n- Step 3: Effort = Work / Productivity rate."
    ),
    llm=llm
)

# Software Engineer Agent
software_engineer_agent = Agent(
    name="Software_Engineer",
    system_prompt=(
        "You are a Software Engineer. Write the complete source code to develop the automation features for CI/CD, security testing integration, and infrastructure provisioning, "
        "based on the design document created by the System Engineer. "
        "Estimate the total lines of code (SLOC) and calculate the effort required using the following steps:"
        "\n- Step 1: Work = Estimate the total number of lines of code (SLOC)."
        "\n- Step 2: Productivity rate = 50 SLOC per day."
        "\n- Step 3: Effort = Work / Productivity rate."
    ),
    llm=llm
)

# Test Engineer Agent
test_engineer_agent = Agent(
    name="Test_Engineer",
    system_prompt=(
        "You are a Test Engineer. Create a detailed test plan document that includes all test cases and execute the test cases based on the software created by the Software Engineer. "
        "Use the design and requirements documents for reference. Estimate the number of test cases and calculate the effort required using the following steps:"
        "\n- Step 1: Work = Estimate the total number of test cases."
        "\n- Step 2: Productivity rate = 2 test cases per day."
        "\n- Step 3: Effort = Work / Productivity rate."
    ),
    llm=llm
)

# Documentation Engineer Agent
documentation_engineer_agent = Agent(
    name="Documentation_Engineer",
    system_prompt=(
        "You are a Documentation Engineer. Create comprehensive user documentation and training materials based on the software, design, and requirements documents. "
        "Estimate the total number of pages and calculate the effort required using the following steps:"
        "\n- Step 1: Work = Estimate the total number of documentation pages."
        "\n- Step 2: Productivity rate = 3 pages per day."
        "\n- Step 3: Effort = Work / Productivity rate."
    ),
    llm=llm
)


C:\Users\SEJAL SRIVASTAV\AppData\Local\Temp\ipykernel_21000\3812591458.py:20: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  self.chain = LLMChain(


In [6]:
project_manager_prompts = {
    "Requirement_Engineer": (
        "I have received the customer requirements from the Customer Proxy. Please list the detailed use cases and requirements for automating a DevSecOps pipeline. "
        "Ensure clarity and completeness in defining each requirement, and make sure they cover CI/CD pipeline automation, security integration, infrastructure-as-code (IaC), and compliance tracking. "
        "Create at least 10 use cases, with a unique name and number each requirement sequentially across all use cases (e.g., REQ-01, REQ-02 for requirements and UC-001, UC-002 for use cases). "
        "Provide detailed estimates for the amount of work, effort, and productivity required to complete the requirements phase, measured by the number of documented use cases and requirements. "
        "Once completed, report back to me so I can proceed with the project plan."
    ),
    "System_Engineer": (
        "The Requirement Engineer has completed documenting the requirements for automating the DevSecOps pipeline. "
        "Please create the detailed design document that includes all design elements, such as architecture for CI/CD, security integration, and infrastructure automation, based on the requirements document created by the Requirement Engineer. "
        "Also, provide detailed estimates for the amount of work, effort, and productivity required to complete the system design phase, measured by the number of pages created for the design document. "
        "Once completed, report back to me so I can proceed with the next steps in the project plan."
    ),
    "Software_Engineer": (
        "The System Engineer has completed the architecture design. "
        "Please proceed with writing the complete source code step-by-step to develop the automation features for CI/CD, security testing integration, and infrastructure provisioning, based on each design element in the design document created by the System Engineer and the requirements document by the Requirement Engineer. "
        "Provide detailed estimates for the amount of work, effort, and productivity required for the development phase, measured by the number of source lines of code (SLOC) written. "
        "Once completed, report back to me so we can proceed with the next steps in the project plan."
    ),
    "Test_Engineer": (
        "The software has been developed. "
        "Please create a detailed test plan document with all test cases and execute them based on the software written by the Software Engineer, the design document created by the System Engineer, and the requirements document created by the Requirement Engineer. "
        "Provide detailed estimates for the amount of work, effort, and productivity required to complete the testing phase, measured by the number of test cases executed. "
        "Once completed, report back to me so we can proceed with the next steps in the project plan."
    ),
    "Documentation_Engineer": (
        "Testing is complete. "
        "Please proceed with creating the user documentation and training materials based on the software, design document, and requirements document for the DevSecOps pipeline. "
        "Also, provide detailed estimates for the amount of work, effort, and productivity required for the documentation phase, measured by the number of documentation pages created. "
        "Once completed, report back to me so we can finalize the project."
    ),
    "Project_Manager": (),
    "Customer_Proxy": ()
}


In [7]:
# Initialize the shared conversation history
conversation_history = ""

# Initial message from Customer Proxy for a DevSecOps pipeline project
initial_message = (
    "I want to create a fully automated DevSecOps pipeline for our organization. "
    "The pipeline should handle continuous integration, delivery, and deployment, integrate security tools like SAST/DAST, "
    "automate infrastructure provisioning using Infrastructure-as-Code (IaC), and ensure compliance and audit logging throughout the process."
)

# Add the initial message to the conversation history
conversation_history += f"Customer_Proxy: {initial_message}\n"
last_message = initial_message
message_count = 1

# Customer Proxy sends message to Project Manager
project_manager_response = project_manager_agent.respond(initial_message, conversation_history=conversation_history)
print(f"{project_manager_agent.name}:\n{project_manager_response}\n")
conversation_history += f"Project_Manager: {project_manager_response}\n"

# Loop to simulate conversation between agents
while True:
    # Customer Proxy responds to Project Manager
    customer_proxy_response = customer_proxy_agent.respond(project_manager_response, conversation_history=conversation_history)
    print(f"{customer_proxy_agent.name}:\n{customer_proxy_response}\n")
    conversation_history += f"Customer_Proxy: {customer_proxy_response}\n"

    user_input = input("Press 'Enter' to continue the conversation, or type 'exit' to end it: ")
    if user_input.lower() == 'exit':
        break

    # Project Manager responds back
    project_manager_response = project_manager_agent.respond(customer_proxy_response, conversation_history=conversation_history)
    print(f"{project_manager_agent.name}:\n{project_manager_response}\n")
    conversation_history += f"Project_Manager: {project_manager_response}\n"

    user_input = input("Press 'Enter' to continue the conversation, or type 'exit' to end it: ")
    if user_input.lower() == 'exit':
        break

    # Requirement Engineer documents use cases and requirements for the DevSecOps pipeline
    req_eng_response = requirement_engineer_agent.respond(
        "Please document the detailed requirements and use cases for the DevSecOps pipeline.",
        conversation_history=conversation_history
    )
    print(f"{requirement_engineer_agent.name}:\n{req_eng_response}\n")
    conversation_history += f"Requirement_Engineer: {req_eng_response}\n"

    user_input = input("Press 'Enter' to continue the conversation, or type 'exit' to end it: ")
    if user_input.lower() == 'exit':
        break

    # System Engineer starts the design phase for the DevSecOps pipeline
    sys_eng_response = system_engineer_agent.respond(
        "Start the design phase for the DevSecOps pipeline based on the documented requirements.",
        conversation_history=conversation_history
    )
    print(f"{system_engineer_agent.name}:\n{sys_eng_response}\n")
    conversation_history += f"System_Engineer: {sys_eng_response}\n"

    user_input = input("Press 'Enter' to continue the conversation, or type 'exit' to end it: ")
    if user_input.lower() == 'exit':
        break

    # Software Engineer begins development
    software_response = software_engineer_agent.respond(
        "Start coding for the DevSecOps pipeline based on the design.",
        conversation_history=conversation_history
    )
    print(f"{software_engineer_agent.name}:\n{software_response}\n")
    conversation_history += f"Software_Engineer: {software_response}\n"

    user_input = input("Press 'Enter' to continue the conversation, or type 'exit' to end it: ")
    if user_input.lower() == 'exit':
        break

    # Test Engineer prepares tests
    test_response = test_engineer_agent.respond(
        "Prepare and execute test cases for the DevSecOps pipeline.",
        conversation_history=conversation_history
    )
    print(f"{test_engineer_agent.name}:\n{test_response}\n")
    conversation_history += f"Test_Engineer: {test_response}\n"
    
    user_input = input("Press 'Enter' to continue the conversation, or type 'exit' to end it: ")
    if user_input.lower() == 'exit':
        break

    # Documentation Engineer prepares user guides
    doc_response = documentation_engineer_agent.respond(
        "Prepare user documentation and training materials for the DevSecOps pipeline.",
        conversation_history=conversation_history
    )
    print(f"{documentation_engineer_agent.name}:\n{doc_response}\n")
    conversation_history += f"Documentation_Engineer: {doc_response}\n"

print("Conversation ended.")


Project_Manager:
Thank you for your requirements. I will create a project plan to develop a fully automated DevSecOps pipeline for your organization. This project will be broken down into several key areas, and I will coordinate with different engineering teams to ensure we meet your expectations.

### Project Plan for Fully Automated DevSecOps Pipeline

#### Project Overview
- **Objective**: Create a fully automated DevSecOps pipeline that handles CI/CD, integrates security tools, automates infrastructure provisioning, and ensures compliance.
- **Timeline**: Estimated completion in 3 months.
- **Key Milestones**:
  1. Requirements Gathering and Analysis
  2. Tool Selection and Architecture Design
  3. Implementation of CI/CD
  4. Integration of Security Tools
  5. Infrastructure Automation
  6. Compliance and Audit Logging
  7. Testing and Validation
  8. Deployment and Monitoring
  9. Documentation and Training
  10. Project Closure

#### Milestone Breakdown

1. **Requirements Gather

Press 'Enter' to continue the conversation, or type 'exit' to end it:  exit


Conversation ended.


In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
import os

# Initialize the language model
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7,
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

class Agent:
    def __init__(self, name, system_prompt, llm):
        self.name = name
        self.llm = llm
        self.system_prompt = system_prompt
        self.chat_prompt = ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template(self.system_prompt),
            HumanMessagePromptTemplate.from_template("{conversation_history}\n{input}")
        ])
        self.chain = LLMChain(
            llm=self.llm,
            prompt=self.chat_prompt,
            verbose=False
        )

    def respond(self, input_text, conversation_history):
        response = self.chain.predict(
            input=input_text,
            conversation_history=conversation_history
        )
        return response

# Define agents with Scrum roles and specific responsibilities
customer_proxy_agent = Agent(
    "Customer_Proxy",
    "Communicate customer's requirements to the Product Owner for the DevSecOps pipeline project.",
    llm
)

product_owner_agent = Agent(
    "Product_Owner",
    "Define and prioritize the backlog items and requirements for the DevSecOps pipeline. Set goals for each sprint.",
    llm
)

scrum_master_agent = Agent(
    "Scrum_Master",
    "Facilitate sprint planning, daily standups, sprint reviews, and retrospectives. Ensure the team focuses on sprint goals.",
    llm
)

requirement_engineer_agent = Agent(
    "Requirement_Engineer",
    "Define and document requirements for the DevSecOps pipeline, including use cases for CI/CD, security integration, and IaC. Provide time estimates based on typical practices.",
    llm
)

system_engineer_agent = Agent(
    "System_Engineer",
    "Create a design document covering architecture for CI/CD, security, and IaC. Provide time estimates for design tasks in DevSecOps.",
    llm
)

software_engineer_agent = Agent(
    "Software_Engineer",
    "Develop software based on design specifications for the DevSecOps pipeline, focusing on CI/CD, security, and IaC. Estimate required time.",
    llm
)

test_engineer_agent = Agent(
    "Test_Engineer",
    "Develop a test plan and execute test cases for the DevSecOps pipeline based on the software, design, and requirements documents. Provide estimated time for testing.",
    llm
)

documentation_engineer_agent = Agent(
    "Documentation_Engineer",
    "Create user documentation and training materials based on project specifications for the DevSecOps pipeline. Estimate documentation time.",
    llm
)

release_manager_agent = Agent(
    "Release_Manager",
    "Coordinate deployment and release activities for the DevSecOps pipeline, ensuring readiness for production. Estimate release management time.",
    llm
)

architect_agent = Agent(
    "Architect",
    "Develop high-level system architecture for the DevSecOps pipeline, ensuring scalability, performance, and security.",
    llm
)

business_analyst_agent = Agent(
    "Business_Analyst",
    "Gather and document business requirements from stakeholders to align with organizational goals for the DevSecOps pipeline.",
    llm
)

# Define the shared conversation history
conversation_history = ""

# Initial message from Customer Proxy
initial_message = (
    "I want to create a fully automated DevSecOps pipeline for our organization. "
    "The pipeline should handle continuous integration, delivery, and deployment, integrate security tools like SAST/DAST, "
    "automate infrastructure provisioning using Infrastructure-as-Code (IaC), and ensure compliance and audit logging throughout the process."
)

# Add initial message to the conversation history
conversation_history += f"Customer_Proxy: {initial_message}\n"
print(f"Customer_Proxy:\n{initial_message}\n")

# Define a function to handle each sprint cycle
def run_sprint(sprint_number):
    global conversation_history
    print(f"\n=== Sprint {sprint_number} ===\n")
    
    # Sprint Planning - Product Owner defines the sprint goal
    sprint_goal = product_owner_agent.respond(
        "Define the sprint goal and prioritize the backlog for the DevSecOps pipeline.",
        conversation_history
    )
    print(f"Product_Owner:\n{sprint_goal}\n")
    conversation_history += f"Product_Owner: {sprint_goal}\n"
    
    # Assign tasks to Development Team members
    development_team_agents = {
        "Requirement_Engineer": requirement_engineer_agent,
        "System_Engineer": system_engineer_agent,
        "Software_Engineer": software_engineer_agent,
        "Test_Engineer": test_engineer_agent,
        "Documentation_Engineer": documentation_engineer_agent,
        "Release_Manager": release_manager_agent,
        "Architect": architect_agent,
        "Business_Analyst": business_analyst_agent
    }

    for role, agent in development_team_agents.items():
        task_message = f"Sprint task for {role}: Work on user stories related to your responsibilities. Provide an estimated time frame for completion."
        response = agent.respond(task_message, conversation_history)
        print(f"{role}:\n{response}\n")
        conversation_history += f"{role}: {response}\n"

    # Daily Standup
    print("\n--- Daily Standup ---")
    for role, agent in development_team_agents.items():
        standup_message = f"Daily Standup for {role}: Share progress, blockers, and next steps."
        standup_response = agent.respond(standup_message, conversation_history)
        print(f"{role}:\n{standup_response}\n")
        conversation_history += f"{role}: {standup_response}\n"

    # Sprint Review - Scrum Master leads the review
    review_message = "Sprint Review: Present completed work and gather feedback from stakeholders."
    review_response = scrum_master_agent.respond(review_message, conversation_history)
    print(f"Scrum_Master:\n{review_response}\n")
    conversation_history += f"Scrum_Master: {review_response}\n"

    # Sprint Retrospective - Scrum Master leads the retrospective
    retrospective_message = "Sprint Retrospective: Discuss what went well, areas for improvement, and any blockers encountered."
    retrospective_response = scrum_master_agent.respond(retrospective_message, conversation_history)
    print(f"Scrum_Master:\n{retrospective_response}\n")
    conversation_history += f"Scrum_Master: {retrospective_response}\n"

# Execute a sprint cycle
run_sprint(sprint_number=1)

print("Sprint cycle completed.")


C:\Users\SEJAL SRIVASTAV\AppData\Local\Temp\ipykernel_21000\1003827170.py:11: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


Customer_Proxy:
I want to create a fully automated DevSecOps pipeline for our organization. The pipeline should handle continuous integration, delivery, and deployment, integrate security tools like SAST/DAST, automate infrastructure provisioning using Infrastructure-as-Code (IaC), and ensure compliance and audit logging throughout the process.


=== Sprint 1 ===

Product_Owner:
To establish a fully automated DevSecOps pipeline, we need to break down the requirements into manageable backlog items and prioritize them for efficient execution. Below is a defined sprint goal along with a prioritized backlog for the initial sprints.

### Sprint Goal:
**Sprint 1:** Establish the foundational pipeline components to support Continuous Integration (CI) and Continuous Delivery (CD) with basic automated testing.

**Sprint 2:** Integrate security tools (SAST & DAST) into the pipeline and ensure the pipeline adheres to security best practices.

**Sprint 3:** Automate infrastructure provisioning usi